In [1]:
import timeit
import math
import numpy as np

# Path Scoring 
- The next procedure is to determine which node we should travel to after the start node.
- The possible nodes that we can travel to are nodes that are "walkable" and dont act as walls that are in the open list.
## Path Scoring
- The node we choose to travel to is the one with the smallest F Score
    
    1.) F-Score = G-Cost + Hueristic (F = G + H)
    
    - G-Cost is the cost of moving from the start node to any other square/node on the board.
        - Cost of 10 to move horozontally or vertically one node, and a cost of 14 to move diagonally to a node.
    - To calculate g-cost of a node add 10 or 14 to the parent nodes g-cost.
    
    
    - H-Cost us the estimated movement cost from the current square to the final destination (end node) also it is important to note that this is an estimation.
        - Calculating H by the Manhattan method. The number of horzontal and ventical nodes from a specific node to an end node multiplied by 10.
    
    
    - F-score is calculated by adding the previous costs together
    
"Our path is generated by repeatedly going through our open list and choosing the square with the lowest F score."
  

In [2]:
# Need to create a node class
'''
Each node contains the following attributes:
- a position on the board. Tuple like (1,1) i.e. second row 
second column element.
- parent node position
- type of node (walkable or unwalkable) boolean
- G-Cost
- H-Cost
- F-Score
'''
class Node:
    '''Node class for algo'''
    
    def __init__(self, y, x, walkable):
        self.y = y
        self.x = x
        self.walkable = walkable
        self.position = None
        self.parent = None
        self.g = 0 
        self.h = 0 
        self.f = 0
      
    '''Creating methods to populate node data and attributes'''
    
    def set_position(self): # sets the nodes position (identity) 
        self.position = (self.y, self.x)
        
    def g_cost(self): # calculates g_cost and sets it to node g attribute
        distance = ((self.x - self.parent.x)**2 + (self.y - self.parent.y)**2)**0.5
        if distance == 1:
            self.g = (self.parent.g + 10)
        else:
            self.g = (self.parent.g + 14)
    
    def h_cost(self, end_node): # calculates h_cost and sets it to node h attribute
        self.h = (abs(self.x - end_node.x) + abs(self.y - end_node.y)) * 10
    
    def f_score(self): # calculates f_score and sets it to node f attribute
        self.f = self.g + self.h
        
        


# Search Area
- The search area should be a simple sqauare grid (2-dimentional array)
- Each element in the 2-dimentional array represents a square/node 
- There is a start node and an end node (square)
- Each node is either considered walkable or not walkable

In [3]:
# The skeleton will be used to display the path traveled
# this will also be used to develope a map of nodes 
# zeros mean "walkable" nodes whereas ones mean "not walkable"

def generate_node_map(skeleton):
    '''
    This function will create a map of nodes based off
    of a two dimentional array of ones and zeros.
    '''
    # Create an empty 2-dimentional array of the same dimentions as the input grid 
    node_map = [[np.nan for _ in range(len(skeleton[0]))] for _ in range(len(skeleton))]
    
    # populate the empty node map with walkable and unwalkable nodes
    # each node will be initialized with a y-coor, x-coor, and whether it is a wall or not (walkable)
    for y in range(len(skeleton)): # iterate rows
        for x in range(len(skeleton[0])): # iterate cols
            node = skeleton[y][x] # get "node"
            if node: # if the node is a one
                # create a non-walkable node and add it in the corresponding
                # position on our node map
                node = Node(y, x, False) 
                node.set_position()
                node_map[y][x] = node
            else: # Otherwise if it is not a one (0) create a walkable node
                # and add it to the corresponding position on our node map
                node = Node(y, x, True)
                node.set_position()
                node_map[y][x] = node
    
    return node_map





In [4]:
def check_surrounding_nodes(arr, node):
    '''
    This function implements not cutting corners. We can only travel
    diagonally when there are no walls in the 4 adjacent positions surrounding 
    the node.
    '''
    
    # Define the possible directions: top, bottom, left, right,
    # top-left, top-right, bottom-left, bottom-right
    
    directions = {"all" : [(0, -1), (-1, 0), (0, 1), (1, 0),
                  (-1, -1), (-1, 1), (1, -1), (1, 1)],
                  "walled" : [(0, -1), (-1, 0), (0, 1), (1, 0)]}
    
    movement_type = "all" # We can move in all directions
    
    for direction in directions["walled"]: # check non-diagonal directions
        # get surrounding nodes y and x values
        y = node.y + direction[0]
        x = node.x + direction[1]
        #check to make sure the surrounding nodes exists and lie within the 
        # bounds of our maze (2-dimentional array) 
        if 0 <= y < len(arr) and 0 <= x < len(arr[0]):
            # if any of these adjacent nodes are not walkable 
            if arr[y][x].walkable == False:
                movement_type = "walled" # change the movement type to walled
                break # then immediately break out of the for-loop
            else: 
                # Otherwise continue. If no "walls" are found
                continue
                # the movement type stays as all directions 
                # thus means we can move diagonally
                
    return directions[movement_type]


In [5]:
def check_parent(node, test_parent,open_list):
    '''
    This function implements altering the parent node
    - Only executed if one of the surrounding nodes of 
    the current node is already in the open_list
    - compares the g-cost between using the existing parent node
    and the current parent node in the g-cost calculation.
    - If the g-cost is lower using the current node as the parent node
    change the nodes parent in the open_list to the current node
    - Otherwise leave it.
    '''
    
    # removes reoccurring node from the open list
    # saves its current parent and g-cost.
    open_list.remove(node)
    current_parent = node.parent
    current_g = node.g
    # make the reoccurring nodes parent the current node
    # calculate new g-cost and set it as its g-attribute 
    node.parent = test_parent
    node.g_cost()
    new_g = node.g
    
    if new_g < current_g: # if the new g-cost (using the current node as parent) is less than the existing g
        return node # return the altered node that has the current node set as its parent 
    else: # Otherwise the original g-cost using its original parent is more efficient
        # reassign the original parent and g-cost
        node.parent = current_parent
        node.g = current_g
        return node
                

In [6]:
def find_valid_nodes(arr, node, open_list, closed_list,dir_func):
    '''
    This function will return the valid nodes and also reassign parent nodes
    Valid Nodes satisfy The following:
    - Adjacent and maybe nodes in diagonal positions
    - Walkable (not walls)
    - not in closed list
    '''
    
    directions = dir_func(arr, node) # returns a list of directions to consider
    
    new_nodes = [] # list that will hold the valid nodes
    # iterate throught the directions returned in dir_func (check_surrounding_nodes)
    for direction in directions:
        y = node.y + direction[0]
        x = node.x + direction[1]
        # make sure the coordinates of the "node" exists in the maze bounds
        if 0 <= y < len(arr) and 0 <= x < len(arr[0]):
            temp_node = arr[y][x]
            # Check to make sure node is not a wall or in the open and closed list
            if (temp_node.walkable == True) and (temp_node not in closed_list) and (temp_node not in open_list):
                temp_node.parent = node
                new_nodes.append(temp_node)
            # If there is a reoccuring node (node that is already in the open_list)
            elif temp_node in open_list:
                temp_node = check_parent(temp_node, node,open_list) # return node that has parent of the lower g-score
                new_nodes.append(temp_node) # append the reoccuring node to the new_nodes list
    
    # return list of the valid nodes with correct parents
    return new_nodes
        

# Starting The Search 
- What we need to to is find the most efficient path from the start node to the end node
- Put simply we start at the start node check the adjeacent squares then keep searching outward from the start node until we find the end node
## Starting The Search
    1.) we need an open list and closed list
    - Open list will contain the all of the nodes that need to be checked out
    - Closed list contains all the squares that have already been checked out and need no further examination
    2.) Add the start node to the open list
    
    3.) Find all the squares surrounding the start node (adjacent squares)
    - Add these adjacent nodes to the open list for further examination.
    - For each adjacent square/node make the start node their "Parent Node".
    4.) Finally drop the start node from the open list and add it to the closed list.
   

In [7]:
def initialize_search(node_map, start_position = (0,0) , end_position = (-1,-1)):
    '''
    This function will initialize the search for the shortest path
    it will take in an array filled with nodes, a start node position
    , and an end node position
    '''
    open_list = [] # list of nodes that are still being examined/considered
    closed_list = [] # nodes that have already been picked/considered
    
    start_node = node_map[start_position[0]][start_position[-1]] # create start node @ start position
    end_node = node_map[end_position[0]][end_position[-1]] # create end node at end position
    
    open_list.append(start_node) # add start node to the open list
    
    return open_list, closed_list, end_node
    

#  Continuing the search 
    1.) Choose the node with lowest F-Score in the open list
       - Drop this specific node from the open list and add it to the closed list.
       
       
    2.) Now identify all of the adjacent squares and add them to the open list if they are not already in the open list, not in the closed list, and not unwalkable.
    - For the newly added squares to the open list make the current square the parent square of these nodes.
    
    
    3.) If an adjacent node has already been on the open list...
    - check to see if the G score for that square is lower if we use the current square to get there. If not, don’t do anything.
    - If the G-Cost of the new path is lower make the current node the parent node for that node
    
    
    4.) Sort open list from lowest to highest f-scores (and choose lowest f-score. 
    
    
Keep repeating this above process until we have found the end node and add it to our closed list



In [8]:
def search(node_map, end_node, open_list, closed_list):
    '''
    Finds the end node
    '''
    
    #Keep doing the following until the end node is in the closed list
    while end_node not in closed_list:
        # calculate scores for each of the nodes in the open-list
        for node in open_list:
            try:
                node.g_cost()
            except Exception as e:
                node.g = 0
                
            node.h_cost(end_node)
            node.f_score()
        
        # sort the open list based on f-scores in ascending order
        open_list.sort(key = lambda x: x.f, reverse = True)
        # initiate a current node (the one with the lowest F-score)
        try:
            current_node = open_list.pop()
        except Exception as e: 
            return False, None
        
        # find the valid nodes surrounding this current node
        valid_nodes = find_valid_nodes(node_map, current_node, open_list, closed_list, check_surrounding_nodes)

        open_list.extend(valid_nodes)
        # add current node to the closed list
        closed_list.append(current_node)
    
    target_node = closed_list.pop()
    return True, target_node
        
        
    


In [9]:
def paint_path(grid, target_node):
    '''
    Once end node is found this function will run
    - Traverses through the parents starting at the end node until
    it hits the start node
    - Paints the optimal path with integers
    - start node = 5
    - end node = 3
    - steps between = 8
    '''
    
    # paint the position of the target node (end node when found is the target node)
    grid[target_node.y][target_node.x] = 3
    current_node = target_node.parent
    
    # While the conditions are met do the following:
    while True:
        if current_node.parent != None:
            grid[current_node.y][current_node.x] = 8
            current_node = current_node.parent
    # Otherwise:
        else:
            # paint final node
            grid[current_node.y][current_node.x] = 5
            # break out of loop
            break

    for row in grid:
        print(row)
    
    
    

In [10]:
def path_finder(grid):
    '''
    Encapsulates the entire pathfinding process into one function
    - Takes in start position and end position
    - generate a map of nodes based on the grid
    - Finds end node.., then paints end node
    - prints no path found if there is no map.
    '''
    
    # user input for start and end position
    start_position = eval(input("Please enter a starting coordinate point: "))
    end_position = eval(input("Please enter an ending coordinate point: "))
    # alter start and end nodes to make sure they wont be walls
    grid[start_position[0]][start_position[-1]] = 0
    grid[end_position[0]][end_position[-1]] = 0
    # create node map
    node_map = generate_node_map(grid)
    # find the end node
    open_list, closed_list, end_node = initialize_search(node_map, start_position, end_position)
    found, target_node = search(node_map, end_node, open_list, closed_list)
    if found: # show the shortest path traveled
        paint_path(grid, target_node)
    else: # no path to show because it doesnt exist
        print("No Path Found")
    


In [11]:
def main():
    
    grid = [
        [0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,1,0,0,1],
        [0,0,0,0,0,0,1,0]
    ]
    
    path_finder(grid)



In [12]:
main()

Please enter a starting coordinate point: (0,0)
Please enter an ending coordinate point: (2,6)
[5, 8, 8, 8, 8, 8, 0, 0]
[0, 0, 0, 0, 1, 0, 8, 0]
[0, 0, 0, 0, 1, 0, 3, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]
